In [2]:
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
import re
from nltk.corpus import stopwords
import string
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
product = pd.read_csv('product_info.csv')
reviews1 = pd.read_csv('reviews_0-250.csv')
reviews2 = pd.read_csv('reviews_250-500.csv')
reviews3 = pd.read_csv('reviews_500-750.csv')
reviews4 = pd.read_csv('reviews_750-1250.csv')
reviews = pd.concat([reviews1, reviews2, reviews3, reviews4])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
/tmp/ipykernel_97/219834932.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews1 = pd.read_csv('reviews_0-250.csv')
/tmp/ipykernel_97/219834932.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews4 = pd.read_csv('reviews_750-1250.csv')


In [3]:
reviews

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119312,119312,9109189891,5,1.0,NaN,0,0,0,2022-04-29,I LOVE this stuff! It works amazingly well at ...,Great product!,NaN,NaN,NaN,NaN,P500101,Hydro Ungrip Makeup Remover + Cleansing Water,MILK MAKEUP,32.0
119313,119313,8115925555,5,1.0,NaN,0,0,0,2022-04-29,love this micellar water from Milk! it removes...,NaN,medium,brown,oily,red,P500101,Hydro Ungrip Makeup Remover + Cleansing Water,MILK MAKEUP,32.0
119314,119314,10723607564,5,1.0,NaN,0,0,0,2022-04-29,I loveeeeee Milk!!! Just discovered this brand...,NaN,mediumTan,brown,dry,black,P500101,Hydro Ungrip Makeup Remover + Cleansing Water,MILK MAKEUP,32.0
119315,119315,5953458355,5,1.0,NaN,0,0,0,2022-04-29,"My new favorite makeup remover. First of all, ...",Works extremely well,light,blue,combination,blonde,P500101,Hydro Ungrip Makeup Remover + Cleansing Water,MILK MAKEUP,32.0


In [14]:
product

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,P467659,Couture Clutch Eyeshadow Palette,1070,Yves Saint Laurent,2790,4.4286,7.0,NaN,NaN,NaN,...,0,0,0,NaN,Makeup,Eye,Eye Palettes,0,NaN,NaN
8490,P500874,L'Homme Eau de Parfum,1070,Yves Saint Laurent,2319,4.6367,556.0,2 oz / 60 mL,Size + Concentration + Formulation,2 oz / 60 mL eau de parfum spray,...,0,0,0,"['Layerable Scent', 'Woody & Earthy Scent']",Fragrance,Men,Cologne,1,129.0,129.0
8491,P504428,Mon Paris Eau de Parfum Gift Set,1070,Yves Saint Laurent,1475,5.0000,2.0,NaN,NaN,NaN,...,1,1,0,NaN,Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
8492,P504448,Y Eau de Parfum Gift Set,1070,Yves Saint Laurent,840,NaN,NaN,NaN,NaN,NaN,...,1,0,0,NaN,Fragrance,Value & Gift Sets,Cologne Gift Sets,0,NaN,NaN


In [18]:
data = reviews[["product_name","review_text","rating"]]

In [20]:
data

,product_name,review_text,rating
0,Gentle Hydra-Gel Face Cleanser,I use this with the Nudestix “Citrus Clean Bal...,5
1,Lip Sleeping Mask Intense Hydration with Vitam...,I bought this lip mask after reading the revie...,1
2,Lip Sleeping Mask Intense Hydration with Vitam...,My review title says it all! I get so excited ...,5
3,Lip Sleeping Mask Intense Hydration with Vitam...,I’ve always loved this formula for a long time...,5
4,Lip Sleeping Mask Intense Hydration with Vitam...,"If you have dry cracked lips, this is a must h...",5
...,...,...,...
119312,Hydro Ungrip Makeup Remover + Cleansing Water,I LOVE this stuff! It works amazingly well at ...,5
119313,Hydro Ungrip Makeup Remover + Cleansing Water,love this micellar water from Milk! it removes...,5
119314,Hydro Ungrip Makeup Remover + Cleansing Water,I loveeeeee Milk!!! Just discovered this brand...,5
119315,Hydro Ungrip Makeup Remover + Cleansing Water,"My new favorite makeup remover. First of all, ...",5


In [24]:
data.rename(columns={"product_name": "product", "review_text": "Review"}, inplace=True)

/tmp/ipykernel_97/466189049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"product_name": "product", "review_text": "Review"}, inplace=True)


In [28]:
data

,product,Review,rating
0,Gentle Hydra-Gel Face Cleanser,I use this with the Nudestix “Citrus Clean Bal...,5
1,Lip Sleeping Mask Intense Hydration with Vitam...,I bought this lip mask after reading the revie...,1
2,Lip Sleeping Mask Intense Hydration with Vitam...,My review title says it all! I get so excited ...,5
3,Lip Sleeping Mask Intense Hydration with Vitam...,I’ve always loved this formula for a long time...,5
4,Lip Sleeping Mask Intense Hydration with Vitam...,"If you have dry cracked lips, this is a must h...",5
...,...,...,...
119312,Hydro Ungrip Makeup Remover + Cleansing Water,I LOVE this stuff! It works amazingly well at ...,5
119313,Hydro Ungrip Makeup Remover + Cleansing Water,love this micellar water from Milk! it removes...,5
119314,Hydro Ungrip Makeup Remover + Cleansing Water,I loveeeeee Milk!!! Just discovered this brand...,5
119315,Hydro Ungrip Makeup Remover + Cleansing Water,"My new favorite makeup remover. First of all, ...",5


In [29]:
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www.\S+', '', text)
    text = re.sub('\+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Review"] = data["Review"].apply(clean)

/tmp/ipykernel_97/388847248.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Review"] = data["Review"].apply(clean)


In [33]:
ratings = data["rating"].value_counts()
numbers = ratings.index
quantity = ratings.values

In [34]:
figure = px.pie(data, values=quantity, names=numbers,hole = 0.5)
figure.show()

In [35]:
figure = px.pie(data, values=quantity, names=numbers,hole = 0.5)
figure.show()

In [36]:
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Review"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Review"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Review"]]
data = data[["Review", "Positive", "Negative", "Neutral"]]
data.head()

/tmp/ipykernel_97/2624820480.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_97/2624820480.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_97/2624820480.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Review,Positive,Negative,Neutral
0,use nudestix “citrus clean balm makeup melt“ ...,0.248,0.040,0.713
1,bought lip mask read review hype unfortun meet...,0.172,0.000,0.828
2,review titl say get excit get bed appli lip ma...,0.081,0.105,0.813
3,i'v alway love formula long time honest don't ...,0.420,0.000,0.580
4,dri crack lip must week use learn alway bedsid...,0.101,0.000,0.899


In [37]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])
def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)

Neutral 🙂 


In [38]:
print("Positive: ", x)
print("Negative: ", y)
print("Neutral: ", z)

Positive:  275283.3090000456
Negative:  40050.03799999971
Neutral:  729089.0479998707


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f00183f9-5f53-4385-a647-c0cb231e91f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>